# Exploratory Text Analysis

## What kinds of text analysis are there?

* analyst knows the pattern
    * regular expressions
* analyst does not know the pattern
    * natural language processing
        * compares historical examples to judge novel cases
            * comparisons are statistical and approximate
            

In [ ]:
# [a-zA-Z]{3}[ ]?[0-9]{4} -> 3 letters of any case, followed by 1 space (optional), followed by 4 nums!
# ?      means 0 or 1 occurance
# +      means 1 or more occurance
# *      means 0 or more occurance
# ^      means start of the pattern
# $      means end of the pattern

### Examples of Analysis

When you know the pattern:

In [1]:
pattern = '£ ?[0-9][0-9]?' # £ then SPACE-optional then digit then digit-optional

document = 'My eggs cost £ 3, bread cost £2, vodka cost £ 35'

In [2]:
import re

In [3]:
re.findall(pattern, document)

['£ 3', '£2', '£ 35']

If you dont:

* sentiment analysis
    * how positive/negative is this (new) review?
* topic analysis
    * what is this document about?

## What can I do if I know what pattern I want to find?

* finding ("extracting")
    * what matches the pattern?
* matching ("validating")
    * does the entire document match YES/NO?
* substitue ("replacing")
    * replace a part that matches a pattern with another...

## How do I validate text with pandas?

In [4]:
import pandas as pd

In [5]:
ti = pd.read_csv('https://raw.githubusercontent.com/a-forty-two/diamler_17_jan/main/titanic.csv')
ti.sample(2)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
230,1,1,female,35.0,1,0,83.475,S,First,woman,False,C,Southampton,yes,False
222,0,3,male,51.0,0,0,8.050,S,Third,man,True,NaN,Southampton,no,True


In [6]:
ti['ticket'] = "Ticket: " + ti['class'] + "; Price: $ " + ti['fare'].astype(str) + "; Port: " + ti['embark_town'] + ";"

In [7]:
ti[['class', 'fare', 'embark_town', 'ticket']].head(3)

,class,fare,embark_town,ticket
0,Third,7.2500,Southampton,Ticket: Third; Price: $ 7.25; Port: Southampton;
1,First,71.2833,Cherbourg,Ticket: First; Price: $ 71.2833; Port: Cherbourg;
2,Third,7.9250,Southampton,Ticket: Third; Price: $ 7.925; Port: Southampton;


In [8]:
pattern = '(First|Second)'

ti['class'].str.match(pattern)

0      False
1       True
2      False
3       True
4      False
       ...  
886     True
887     True
888    False
889     True
890    False
Name: class, Length: 891, dtype: bool

In [9]:
ti.loc[ ti['class'].str.match(pattern)  , 'survived'].mean()

0.5575

In [10]:
pattern = '(Third)'
ti.loc[ ti['class'].str.match(pattern)  , 'survived'].mean()

0.24236252545824846

## How do I extract data with pandas?

In [11]:
ti[['class', 'fare', 'embark_town', 'ticket']].head(3)

,class,fare,embark_town,ticket
0,Third,7.2500,Southampton,Ticket: Third; Price: $ 7.25; Port: Southampton;
1,First,71.2833,Cherbourg,Ticket: First; Price: $ 71.2833; Port: Cherbourg;
2,Third,7.9250,Southampton,Ticket: Third; Price: $ 7.925; Port: Southampton;


In [12]:
#just for demo, no real purpose
pattern = '([0-9.]+)'

ti['ticket'].str.extract(pattern).sample(4)

,0
244,7.225
677,9.8417
538,14.5
126,7.75


## How do I substitue text with pandas?

In [13]:
ti['ticket'].str.replace('$', '€').sample(1)

372    Ticket: Third; Price: € 8.05; Port: Southampton;
Name: ticket, dtype: object

## What are regular expressions?

Regular expressions are a language for describing patterns in text.

They are separate from python, but may be used within python program. (And elsewhere, eg., often in SQL).

They are notoriously difficult to read and write; and as a separate language, an additional tool to learn.

## What regular expression patterns can I use?

* literals
    * `a`, find me an `a`
    * `£`, find `£`
    * `!` means `!`
    * ... most symbols mean "find me"
* `.`
    * find any **single** symbol
* character classes -- find a **single** symbol
    * `[abc]` $\rightarrow$ **either** a, b, c
    * `[0-9]` $\rightarrow$ **either** 0, 1, 2, 3,...9
    * `[A-Z]` $\rightarrow$ **either** capital A, B, ... Z
    * inversions
        * `[^abc]` $\rightarrow$ **is not** `a` OR `b` OR `c`
        * `[^a-zA-Z0-9 ]`  $\rightarrow$ **is not** alphanumeric-ish
    
* alternatives -- find the character**s** given by...
    * `(May|June|July)`  $\rightarrow$ **the whole worlds** May OR June..
    

In [14]:
ti['ticket'].str.extract('(Ticket: (First|Second))')

,0,1
0,NaN,NaN
1,Ticket: First,First
2,NaN,NaN
3,Ticket: First,First
4,NaN,NaN
...,...,...
886,Ticket: Second,Second
887,Ticket: First,First
888,NaN,NaN
889,Ticket: First,First


In [15]:
ti['ticket'].str.extract('( [0-9][0-9])')

,0
0,NaN
1,71
2,NaN
3,53
4,NaN
...,...
886,13
887,30
888,23
889,30


In [16]:
ti['ticket'].sample(1)

564    Ticket: Third; Price: $ 8.05; Port: Southampton;
Name: ticket, dtype: object

In [17]:
ti['ticket'].str.extract('(Ticket: [A-Z])').sample(2)

,0
343,Ticket: S
592,Ticket: T


In [18]:
ti['ticket'].str.extract('(T........)').sample(3)

,0
168,Ticket: F
139,Ticket: F
650,Ticket: T


In [19]:
ti['ticket'].str.extract('(Price: [^0-9A-Za-z] ..)').sample(3)

,0
243,Price: $ 7.
538,Price: $ 14
588,Price: $ 8.


In [20]:
ti['ticket'].str.extract('(Port: (Cherbourg|Southampton))').sample(3)

,0,1
733,Port: Southampton,Southampton
12,Port: Southampton,Southampton
340,Port: Southampton,Southampton


* repetitions
    * optional `?`
        * an optional number: `[0-9]?`
    * one or more `+`
        * one or more spaces: ` +`  
    * optional, or more, `*`
        * ` [0-9][0-9]?.[0-9]*`
    

In [21]:
ti['ticket'].str.extract('([0-9][0-9]?.[0-9]*)').sample(3)

,0
211,21.0
735,16.1
731,18.7875


In [22]:
ti['ticket'].str.extract('(Ticket: [a-zA-Z]+)').sample(3)

,0
81,Ticket: Third
706,Ticket: Second
293,Ticket: Third


In [23]:
row = 0
match = 1 # second match

ti['ticket'].str.extractall('([a-zA-Z]+: [a-zA-Z]+)').loc[row, match]

0    Port: Southampton
Name: (0, 1), dtype: object

In [24]:
ti['ticket'].str.extract('([a-zA-Z]+tow?n)')

,0
0,Southampton
1,NaN
2,Southampton
3,Southampton
4,Southampton
...,...
886,Southampton
887,Southampton
888,Southampton
889,NaN


* EXTRA:
    * escaping
        * How do I say, literally, the `.` symbol?
        * `\.`
    

In [25]:
ti['ticket'].str.extract('(\$ [0-9]+\.[0-9]+)').sample(2)

,0
496,$ 78.2667
487,$ 29.7


* positional matching
    * `^` means **at the beginning**
    * `$` means **at the end**

In [26]:
ti['ticket'].str.extractall('([a-zA-Z]+: [a-zA-Z]+;$)').sample(1)

,,0
,match,
209,0,Port: Cherbourg;


In [ ]:
ti['ticket'].str.extractall('(^[a-zA-Z]+: [a-zA-Z]+;)').sample(1)

,,0
,match,
2,0,Ticket: Third;


## Next Steps

* review a "Regex Cheat Sheet"
    * also, eg., https://en.wikipedia.org/wiki/Regular_expression#Examples

## Exercise (30 min)

* find all the words in the tickets
    * HINT: a word is a repeated letter followed by a space or a colon
    * HINT: `[ :]` means a space or a colon
* find all the USD prices
    * HINT: ``` \$ ``` and repeated numbers
    
* find all the high-price tickets
    * HINT: consider `\$`, tripple-digit number, `\.`

In [ ]:
#Solution

In [27]:
ti['ticket'].str.findall('([a-zA-Z]+[ :])').sample(10)

192    [Ticket:, Price:, Port:]
361    [Ticket:, Price:, Port:]
674    [Ticket:, Price:, Port:]
165    [Ticket:, Price:, Port:]
63     [Ticket:, Price:, Port:]
144    [Ticket:, Price:, Port:]
861    [Ticket:, Price:, Port:]
157    [Ticket:, Price:, Port:]
430    [Ticket:, Price:, Port:]
131    [Ticket:, Price:, Port:]
Name: ticket, dtype: object

In [28]:
ti['ticket'].str.extract('(\$ [0-9]+\.[0-9]+)').sample(2)

,0
756,$ 7.7958
799,$ 24.15


In [29]:
row = 0
match = 1 # second match
ti['ticket'].str.extractall('(\$ [0-9][0-9][0-9]+\.[0-9]+)')#.loc[row, 0]

,,0
,match,
27,0,$ 263.0
31,0,$ 146.5208
88,0,$ 263.0
118,0,$ 247.5208
195,0,$ 146.5208
215,0,$ 113.275
258,0,$ 512.3292
268,0,$ 153.4625
269,0,$ 135.6333
